## Pryngles pixx external module

In [190]:
import pickle
import numpy as np
import pandas as pd
import os

You must first download file `reflection-test.pkl` from Google Drive and place it at `/tmp`

In [160]:
f=open("/tmp/reflection-test.pkl.2","rb")
save_values=pickle.load(f)
f.close()

In [224]:
matrices=dict()
columns=["delazim","beta","eta","zeta","nmugs","nmat","nfou","af","S1","S2","S3","S4"]
for i,save in enumerate(save_values):
    args=save["args"]
    obj=save["obj"].replace(" ","_")
    """
    args are:
        0: Number of points (int)
        1: Delta azim (array Nx1)
        2: betas (array: Nx1)
        3: etas (array: Nx1)
        4: zetas
        5: nmugs
        6: nmat
        7: nfou
        8: xmu
        9: rfou
        10: af
    """
    
    #Read properties
    npoints=args[0]
    delta_azim=args[1]
    betas=args[2]
    etas=args[3]
    zetas=args[4]
    nmugs,nmat,nfou=args[5:8]
    xmu=args[8]
    rfou=args[9]
    nx,ny,nz=rfou.shape
    af=args[10]
    hash_mat=hash(rfou.sum())
    
    #Read stokes
    stokes=save["stokes"]
    
    #Save in dataframe
    data=pd.DataFrame(columns=columns)
    data["delazim"]=pd.Series(delta_azim)
    data["beta"]=pd.Series(betas)
    data["eta"]=pd.Series(etas)
    data["zeta"]=pd.Series(zetas)
    data["nmugs"]=nmugs
    data["nmat"]=nmat
    data["nfou"]=nfou
    data["af"]=pd.Series(af)
    data[["S1","S2","S3","S4"]]=pd.DataFrame(stokes)
    
    #Save matrix
    if not hash_mat in matrices.keys():    
        #Save matrix
        ntot=nx*ny*nz
        filename=f"{obj}-{ntot}_{nx}_{ny}_{nz}.mat"
        print(f"Saving file {filename}...")
        np.savetxt(filename,rfou.reshape(ntot,1))

        filename=f"{obj}-xmu-{len(xmu)}.mat"
        print(f"Saving file {filename}...")
        np.savetxt(filename,xmu)

        matrices[hash_mat]=dict(
            obj=obj,
            data=pd.DataFrame(columns=columns),
            rfou=rfou,
        )
    
    #Store data
    matrices[hash_mat]["data"]=pd.concat((matrices[hash_mat]["data"],data),ignore_index=True)
    #break
        
print("Done.")

Saving file planet-5292_63_21_4.mat...
Saving file planet-xmu-21.mat...
Saving file ring_forward-414000_60_20_345.mat...
Saving file ring_forward-xmu-20.mat...
Saving file ring_back-414000_60_20_345.mat...
Saving file ring_back-xmu-20.mat...
Done.


Save matrices:

In [225]:
for hash_mat,item in matrices.items():
    obj=item["obj"]
    print(f"Saving interpolation matrix for {obj}")
    data=item["data"]
    print(f"Original number of interpolation points {len(data)}")
    data_clean=data.drop_duplicates()
    print(f"Number of interpolation points after removing duplicates {len(data_clean)}")
    filename=f"{obj}-interpolation.mat"
    print(f"Saving file {filename}...")
    np.savetxt(filename,data_clean)
    break

Saving interpolation matrix for planet
Original number of interpolation points 696730
Number of interpolation points after removing duplicates 193758
Saving file planet-interpolation.mat...


In [226]:
data_clean.iloc[5000]

delazim    0.952358
beta       0.072198
eta        0.854523
zeta       0.025097
nmugs            21
nmat              3
nfou              3
af         0.000094
S1         0.000001
S2             -0.0
S3             -0.0
S4           0.3277
Name: 5000, dtype: object

## Check using pixx

Read data:

In [227]:
!ls *.mat

planet-21.mat                     ring_back-interpolation.mat
planet-5292_63_21_4.mat           ring_back-xmu-20.mat
planet-interpolation.mat          ring_forward-20.mat
planet-xmu-21.mat                 ring_forward-414000_60_20_345.mat
ring_back-20.mat                  ring_forward-interpolation.mat
ring_back-414000_60_20_345.mat    ring_forward-xmu-20.mat


In [233]:
filename="planet-5292_63_21_4.mat"
ntot,nx,ny,nz=[int(v) for v in filename.split("-")[1].split(".")[0].split("_")]
rfou=np.loadtxt(filename).reshape(nx,ny,nz)
filename="planet-interpolation.mat"
interpolation=np.loadtxt(filename)
filename="planet-xmu-21.mat"
xmu=np.loadtxt(filename)
#Compar
#rfou_ori=matrices[401387150607385127]["rfou"]
#abs(rfou-rfou_ori).mean()

In [213]:
interpolation[5000]

array([ 9.52358144e-01,  7.21983762e-02,  8.54522910e-01,  2.50967097e-02,
        2.10000000e+01,  3.00000000e+00,  3.00000000e+00,  9.43232879e-05,
        1.47379372e-06, -3.41565262e-07, -3.41446062e-07,  3.27699826e-01])

## How reshape works

In [166]:
matrix=np.array([[[1,2,3],[4,5,6]],[[-1,-2,-3],[-4,-5,-6]]])

In [167]:
matrix.shape

(2, 2, 3)

In [168]:
matrix.reshape(12,1)

array([[ 1],
       [ 2],
       [ 3],
       [ 4],
       [ 5],
       [ 6],
       [-1],
       [-2],
       [-3],
       [-4],
       [-5],
       [-6]])

In [169]:
nx,ny,nz=rfou.shape
ntot=nx*ny*nz

In [170]:
rfou.reshape(ntot,1)

array([[ 4.1593132e+01],
       [-2.8785500e-04],
       [ 6.0632031e+00],
       ...,
       [ 0.0000000e+00],
       [ 0.0000000e+00],
       [ 0.0000000e+00]])

In [171]:
rfou[0][0]

array([ 4.1593132e+01, -2.8785500e-04,  6.0632031e+00,  0.0000000e+00])